### Trabalho individual 1
João Pedro Rolim Dias, nº 110305, CDB1

In [1]:
# Setup inicial e import de bibliotecas utilizadas no projeto
import pandas as pd
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, GLPK
import numpy as np

---
Variáveis de decisão: 
- x1 - número de centenas de kits básicos a enviar.
- x2 - número de centenas de kits avançados a enviar.
- x3 - número de centenas de kits premium a enviar.
---
- Custo (C) na FO em milhares de €.
- Em rigor deveria ser um problema de programação linear inteira, porque não há menção as meios kits, mas para efeitos de simplificação da formulação vamos considerar como um problema de programação linear. 

## a) Formulação do problema em programação linear
##### $\min C = 30x_1 + 35x_2 + 105x_3$ (objetivo 1 de minimizar o custo)
##### $\max K = x_1 + x_2 + x_3$ (objetivo 2 de maximizar o envio dos kits)

#####   s.a.: $x_3 \geq 30$ (restrição de enviar pelo menos os 3000 kits premium, ajustada ao facto de estarmos a lidar com centenas)
#####         $3000x_1 + 3500x_2 + 5400x_3 \geq 2100000$ (restrição de ajudar pelo menos 2.1 milhões de habitantes do país)
#####         $12x_1 + 18x_2 + 22x_3 \leq 10000$ (restrição de transporte, ajustada ao facto de a informação de cada kit ser apresentada em toneladas)
#####         $x_1, x_2 \geq 0$ (restrição de kits não negativos)

## b) Verificação de propostas admissíveis e de soluções dominadas

In [2]:
# Função para verificar se uma proposta é admissível
def verificar_admissibilidade(x1, x2, x3):
    # Verificar se atende a todas as restrições
    return (3000*x1 + 3500*x2 + 5400*x3 >= 2100000) and (x3 >= 30) and (12*x1 + 18*x2 + 22*x3 <= 10000) and (x1 >= 0) and (x2 >= 0)

# Função para calcular o número total de kits enviados em uma proposta
def calcular_total_kits(x1, x2, x3):
    return x1 + x2 + x3

# Função para calcular o custo total de uma proposta (em milhares de euros)
def calcular_custo_total(x1, x2, x3):
    return 30*x1 + 35*x2 + 105*x3

# Lista de propostas
propostas = [
    {"nome": "Proposta 1", "kits_basicos": 184, "kits_avancados": 396, "kits_premium": 30},
    {"nome": "Proposta 2", "kits_basicos": 646, "kits_avancados": 0, "kits_premium": 30},
    {"nome": "Proposta 3", "kits_basicos": 761, "kits_avancados": 4, "kits_premium": 30},
    {"nome": "Proposta 4", "kits_basicos": 765, "kits_avancados": 0, "kits_premium": 30}
]

# Verificar admissibilidade, calcular custo total (em milhares de euros) e número total de kits enviados para cada proposta
for proposta in propostas:
    custo_total = calcular_custo_total(proposta["kits_basicos"], proposta["kits_avancados"], proposta["kits_premium"])
    admissivel = verificar_admissibilidade(proposta["kits_basicos"], proposta["kits_avancados"], proposta["kits_premium"])
    total_kits = calcular_total_kits(proposta["kits_basicos"], proposta["kits_avancados"], proposta["kits_premium"])
    
    print("Proposta:", proposta["nome"])
    print("Admissível:", admissivel)
    print("Custo Total:", custo_total)
    print("Centenas de Kits Enviados:", total_kits)
    print("-")

Proposta: Proposta 1
Admissível: True
Custo Total: 22530
Centenas de Kits Enviados: 610
-
Proposta: Proposta 2
Admissível: True
Custo Total: 22530
Centenas de Kits Enviados: 676
-
Proposta: Proposta 3
Admissível: True
Custo Total: 26120
Centenas de Kits Enviados: 795
-
Proposta: Proposta 4
Admissível: True
Custo Total: 26100
Centenas de Kits Enviados: 795
-


##### Principais conclusões da alínea b): 
- São todas soluções admissíveis porque respeitam todas as restrições; 
- A proposta 2 domina a proposta 1, porque permite enviar mais kits com o mesmo custo; 
- A proposta 4 domina a proposta 3 porque permite enviar o mesmo número de kits a um custo mais baixo.

## c) Quantos kits básicos, avançados e premium deve a organização enviar para o país se a organização estiver interessada, exclusivamente, num dos objetivos? É possível atingir os dois objetivos em simultâneo? 

### (i) Minimização do custo da ajuda humanitária

##### $\min C = 30x_1 + 35x_2 + 105x_3$ 

#####   s.a.: $x_3 \geq 30$ 
#####         $3000x_1 + 3500x_2 + 5400x_3 \geq 2100000$ 
#####         $12x_1 + 18x_2 + 22x_3 \leq 10000$ 
#####         $x_1, x_2 \geq 0$ 

In [3]:
# Criar o modelo
model = LpProblem(name="min_custo_ajuda", sense=LpMinimize)
#
# Inicializar as variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat='Integer') for i in range(1, 4)}
#
# Adicionar as restrições
model += (x[3] >= 30, "Min_Kits_Premium")
model += (3000 * x[1] + 3500 * x[2] + 5400 * x[3] >= 2100000, "Min_Ajuda_Habitantes")
model += (12 * x[1] + 18 * x[2] + 22 * x[3] <= 10000, "Max_Peso_Transporte")
#
# Adicionar a função objetivo
obj_func = 30 * x[1] + 35 * x[2] + 105 * x[3]
model += obj_func
# 
# Formulação (visualização do modelo matemático de forma a conferir os dados)
model

min_custo_ajuda:
MINIMIZE
30*x1 + 35*x2 + 105*x3 + 0
SUBJECT TO
Min_Kits_Premium: x3 >= 30

Min_Ajuda_Habitantes: 3000 x1 + 3500 x2 + 5400 x3 >= 2100000

Max_Peso_Transporte: 12 x1 + 18 x2 + 22 x3 <= 10000

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer

In [4]:
status = model.solve() 

# Valor óptimo de problema
model.objective.value()
print(f"objective: {model.objective.value()}")
#
# Solução óptima
# Valores óptimos das variáveis de decisão
#
for var in x.values():
    print(f"{var.name}: {var.value()}")
#
# Valores das variáveis de desvio
#
for name, constraint in model.constraints.items():
    print (f"{name}: {constraint.value()}") 

objective: 22530.0
x1: 646.0
x2: 0.0
x3: 30.0
Min_Kits_Premium: 0.0
Min_Ajuda_Habitantes: 0.0
Max_Peso_Transporte: -1588.0


### (ii) Maximização do total de kits enviados

##### $\max K = x_1 + x_2 + x_3$

#####   s.a.: $x_3 \geq 30$ 
#####         $3000x_1 + 3500x_2 + 5400x_3 \geq 2100000$ 
#####         $12x_1 + 18x_2 + 22x_3 \leq 10000$ 
#####         $x_1, x_2 \geq 0$ 

In [5]:
# Criar o modelo
model = LpProblem(name="max_kits", sense=LpMaximize)
#
# Inicializar as variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat='Integer') for i in range(1, 4)}
#
# Adicionar as restrições
model += (x[3] >= 30, "Min_Kits_Premium")
model += (3000 * x[1] + 3500 * x[2] + 5400 * x[3] >= 2100000, "Min_Ajuda_Habitantes")
model += (12 * x[1] + 18 * x[2] + 22 * x[3] <= 10000, "Max_Peso_Transporte")
#
# Adicionar a função objetivo
obj_func = x[1] + x[2] + x[3]
model += obj_func
# 
# Formulação (visualização do modelo matemático de forma a conferir os dados)
model

max_kits:
MAXIMIZE
1*x1 + 1*x2 + 1*x3 + 0
SUBJECT TO
Min_Kits_Premium: x3 >= 30

Min_Ajuda_Habitantes: 3000 x1 + 3500 x2 + 5400 x3 >= 2100000

Max_Peso_Transporte: 12 x1 + 18 x2 + 22 x3 <= 10000

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer

In [6]:
status = model.solve() 

# Valor óptimo de problema
model.objective.value()
print(f"objective: {model.objective.value()}")
#
# Solução óptima
# Valores óptimos das variáveis de decisão
#
for var in x.values():
    print(f"{var.name}: {var.value()}")
#
# Valores das variáveis de desvio
#
for name, constraint in model.constraints.items():
    print (f"{name}: {constraint.value()}") 

objective: 808.0
x1: 778.0
x2: 0.0
x3: 30.0
Min_Kits_Premium: 0.0
Min_Ajuda_Habitantes: 396000.0
Max_Peso_Transporte: -4.0


## d) Admitindo que a organização atribui igual importância aos objetivos, determine uma solução de envio de kits que garanta: ▪ um custo de envio de aproximadamente 22.5 milhões de euros; e ▪ o envio de aproximadamente 80800 kits.

### Abordagem não preemptiva
#### Minimização da Soma dos Desvios Percentuais Ponderados
#### Formulação em PL por metas

##### $\min Z = \frac{P_1^{-} * d_1^{-} + P_1^{+} * d_1^{+}}{22500} + \frac{P_2^{-} * d_2^{-} + P_2^{+} * d_2^{+}}{808}$

#####   s.a.: $x_3 \geq 30$ 
#####         $3000x_1 + 3500x_2 + 5400x_3 \geq 2100000$ 
#####         $12x_1 + 18x_2 + 22x_3 \leq 10000$ 
#####         $30x_1 + 35x_2 + 105x_3 + d_1^{-} - d_1^{+} = 22500 $ 
#####         $x_1 + x_2 + x_3 + d_2^{-} - d_2^{+} = 808 $ 
#####         $x_1, x_2 \geq 0$ 
#####         $d_1^{-}, d_1^{+}, d_2^{-}, d_2^{+} \geq 0$

In [7]:
# Criar o modelo
model = LpProblem(name="ajuda_pesos_iguais", sense=LpMinimize)
#
# Inicializar as variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 4)}
dm_cost = LpVariable('dm_cost', lowBound=0)
dM_cost = LpVariable('dM_cost', lowBound=0)
dm_kits = LpVariable('dm_kits', lowBound=0)
dM_kits = LpVariable('dM_kits', lowBound=0)
#
# Adicionar as restrições
model += (x[3] >= 30, "Min_Kits_Premium")
model += (3000 * x[1] + 3500 * x[2] + 5400 * x[3] >= 2100000, "Min_Ajuda_Habitantes")
model += (12 * x[1] + 18 * x[2] + 22 * x[3] <= 10000, "Max_Peso_Transporte")
model += (30 * x[1] + 35 * x[2] + 105 * x[3] + dm_cost - dM_cost == 22500, "Meta_Custo")
model += (x[1] + x[2] + x[3] + dm_kits - dM_kits == 808, "Meta_Kits")
#
# Adicionar a função objetivo
meta = np.array([22500, 808])
peso = np.array([1,1])
obj_func = (peso[0]/meta[0]) * (dm_cost + dM_cost) + (peso[1]/meta[1])* (dm_kits + dM_kits)
model += obj_func
# 
# Formulação (visualização do modelo matemático de forma a conferir os dados)
model

ajuda_pesos_iguais:
MINIMIZE
4.4444444444444447e-05*dM_cost + 0.0012376237623762376*dM_kits + 4.4444444444444447e-05*dm_cost + 0.0012376237623762376*dm_kits + 0.0
SUBJECT TO
Min_Kits_Premium: x3 >= 30

Min_Ajuda_Habitantes: 3000 x1 + 3500 x2 + 5400 x3 >= 2100000

Max_Peso_Transporte: 12 x1 + 18 x2 + 22 x3 <= 10000

Meta_Custo: - dM_cost + dm_cost + 30 x1 + 35 x2 + 105 x3 = 22500

Meta_Kits: - dM_kits + dm_kits + x1 + x2 + x3 = 808

VARIABLES
dM_cost Continuous
dM_kits Continuous
dm_cost Continuous
dm_kits Continuous
x1 Continuous
x2 Continuous
x3 Continuous

In [8]:
status = model.solve() 

# Valor óptimo de problema
model.objective.value()
print(f"objective: {model.objective.value()}")
#
# Solução óptima
# Valores óptimos das variáveis de decisão
#
for var in x.values():
    print(f"{var.name}: {var.value()}")
print(f"{dm_cost}: {dm_cost.value()}")
print(f"{dM_cost}: {dM_cost.value()}")
print(f"{dm_kits}: {dm_kits.value()}")
print(f"{dM_kits}: {dM_kits.value()}")
#
# Valores das variáveis de desvio
#
for name, constraint in model.constraints.items():
    print (f"{name}: {constraint.value()}") 

objective: 0.1646996699669967
x1: 646.0
x2: 0.0
x3: 30.0
dm_cost: 0.0
dM_cost: 30.0
dm_kits: 132.0
dM_kits: 0.0
Min_Kits_Premium: 0.0
Min_Ajuda_Habitantes: 0.0
Max_Peso_Transporte: -1588.0
Meta_Custo: 0.0
Meta_Kits: 0.0


## e) Suponha que a organização decidiu dar mais importância ao cumprimento do nível de aspiração do total de kits enviado. Deste modo, atribuiu uma penalização de 8 pontos por cada 10 kits abaixo do nível de aspiração (80800 kits) e uma penalização de 1 ponto por cada milhão de euros acima do nível de aspiração (22.5 milhões de euros)

Deve considerar-se uma abordagem preemptiva já que são dados diferentes níveis de prioridade aos objetivos.
Para calcular os pesos: 
- peso dos kits: 8*100/10
- peso do custo: 1/(1000000 * 1000)

#### Formulação do problema: 
##### $Lex \min Z = \left\{P_2^{-}d_2^{-} + P_2^{+}d_2^{+}, P_1^{-}d_1^{-} + P_1^{+}d_1^{+} \right\}$

##### **Primeiro nível**
#####   $ \min Z = P_2^{-}d_2^{-} + P_2^{+}d_2^{+}$
#####   s.a.: $x_3 \geq 30$ 
#####         $3000x_1 + 3500x_2 + 5400x_3 \geq 2100000$ 
#####         $12x_1 + 18x_2 + 22x_3 \leq 10000$ 
#####         $x_1 + x_2 + x_3 + d_2^{-} - d_2^{+} = 808 $ 
#####         $x_1, x_2 \geq 0$ 
#####         $d_2^{-}, d_2^{+} \geq 0$

In [9]:
# Para o primeiro nível:
# Criar o modelo
model1 = LpProblem(name="preemptiva_e1", sense=LpMinimize)
#
# Inicializar as variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat='Integer') for i in range(1, 4)}
dm_kits = LpVariable('dm_kits', lowBound=0)
dM_kits = LpVariable('dM_kits', lowBound=0)
#
# Adicionar as restrições
model1 += (x[3] >= 30, "Min_Kits_Premium")
model1 += (3000 * x[1] + 3500 * x[2] + 5400 * x[3] >= 2100000, "Min_Ajuda_Habitantes")
model1 += (12 * x[1] + 18 * x[2] + 22 * x[3] <= 10000, "Max_Peso_Transporte")
model1 += (x[1] + x[2] + x[3] + dm_kits - dM_kits == 808, "Meta_Kits")
#
# Adicionar a função objetivo
meta = np.array([808,22500])
peso = np.array([(8*100/10),1,1,(1/(1000000 * 1000))])
obj_func =  (peso[0]*dm_kits)/meta[0] + (peso[1]*dM_kits)/meta[0]
model1 += obj_func
# 
# Formulação (visualização do modelo matemático de forma a conferir os dados)
model1

preemptiva_e1:
MINIMIZE
0.0012376237623762376*dM_kits + 0.09900990099009901*dm_kits + 0.0
SUBJECT TO
Min_Kits_Premium: x3 >= 30

Min_Ajuda_Habitantes: 3000 x1 + 3500 x2 + 5400 x3 >= 2100000

Max_Peso_Transporte: 12 x1 + 18 x2 + 22 x3 <= 10000

Meta_Kits: - dM_kits + dm_kits + x1 + x2 + x3 = 808

VARIABLES
dM_kits Continuous
dm_kits Continuous
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer

In [10]:
status = model1.solve() 

# Valor óptimo de problema
model1.objective.value()
print(f"objective: {model1.objective.value()}")
#
# Solução óptima
# Valores óptimos das variáveis de decisão
#
for var in x.values():
    print(f"{var.name}: {var.value()}")
print(f"{dm_kits}: {dm_kits.value()}")
print(f"{dM_kits}: {dM_kits.value()}")
#
# Valores das variáveis de desvio
#
for name, constraint in model1.constraints.items():
    print (f"{name}: {constraint.value()}") 

objective: 0.0
x1: 778.0
x2: 0.0
x3: 30.0
dm_kits: 0.0
dM_kits: 0.0
Min_Kits_Premium: 0.0
Min_Ajuda_Habitantes: 396000.0
Max_Peso_Transporte: -4.0
Meta_Kits: 0.0


##### **Segundo nível**
#####   $ \min Z = P_1^{-}d_1^{-} + P_1^{+}d_1^{+}$
#####   s.a.: $x_3 \geq 30$ 
#####         $3000x_1 + 3500x_2 + 5400x_3 \geq 2100000$ 
#####         $12x_1 + 18x_2 + 22x_3 \leq 10000$ 
#####         $x_1 + x_2 + x_3 + d_2^{-} - d_2^{+} = 808 $ 
#####         $30x_1 + 35x_2 + 105x_3 + d_1^{-} - d_1^{+} = 22500 $ 
#####         $x_1, x_2 \geq 0$ 
#####         $d_1^{-}, d_1^{+}, d_2^{-}, d_2^{+} \geq 0$

In [11]:
# Para o segundo nível:
# Criar o modelo
model2 = LpProblem(name="preemptiva_e2", sense=LpMinimize)
#
# Inicializar as variáveis de decisão
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat='Integer') for i in range(1, 4)}
dm_cost = LpVariable('dm_cost', lowBound=0)
dM_cost = LpVariable('dM_cost', lowBound=0)
dm_kits = LpVariable('dm_kits', lowBound=0)
dM_kits = LpVariable('dM_kits', lowBound=0)
#
# Adicionar as restrições
model2 += (x[3] >= 30, "Min_Kits_Premium")
model2 += (3000 * x[1] + 3500 * x[2] + 5400 * x[3] >= 2100000, "Min_Ajuda_Habitantes")
model2 += (12 * x[1] + 18 * x[2] + 22 * x[3] <= 10000, "Max_Peso_Transporte")
model2 += (30 * x[1] + 35 * x[2] + 105 * x[3] + dm_cost - dM_cost == 22500, "Meta_Custo")
model2 += (x[1] + x[2] + x[3] + dm_kits - dM_kits == 808, "Meta_Kits")
#
# Adicionar a função objetivo
meta = np.array([808, 22500])
peso = np.array([(8*100/10),1,1,(1/(1000000 * 1000))])
obj_func = (peso[0]*dm_kits)/meta[0] + (peso[1]*dM_kits)/meta[0] + (peso[2]*dm_cost)/meta[1] + (peso[3]*dM_cost)/meta[1]
model2 += obj_func
# 
# Formulação (visualização do modelo matemático de forma a conferir os dados)
model2

preemptiva_e2:
MINIMIZE
4.444444444444445e-14*dM_cost + 0.0012376237623762376*dM_kits + 4.4444444444444447e-05*dm_cost + 0.09900990099009901*dm_kits + 0.0
SUBJECT TO
Min_Kits_Premium: x3 >= 30

Min_Ajuda_Habitantes: 3000 x1 + 3500 x2 + 5400 x3 >= 2100000

Max_Peso_Transporte: 12 x1 + 18 x2 + 22 x3 <= 10000

Meta_Custo: - dM_cost + dm_cost + 30 x1 + 35 x2 + 105 x3 = 22500

Meta_Kits: - dM_kits + dm_kits + x1 + x2 + x3 = 808

VARIABLES
dM_cost Continuous
dM_kits Continuous
dm_cost Continuous
dm_kits Continuous
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer

In [12]:
status = model2.solve() 

# Valor óptimo de problema
model2.objective.value()
print(f"objective: {model2.objective.value()}")
#
# Solução óptima
# Valores óptimos das variáveis de decisão
#
for var in x.values():
    print(f"{var.name}: {var.value()}")
print(f"{dm_cost}: {dm_cost.value()}")
print(f"{dM_cost}: {dM_cost.value()}")
print(f"{dm_kits}: {dm_kits.value()}")
print(f"{dM_kits}: {dM_kits.value()}")
#
# Valores das variáveis de desvio
#
for name, constraint in model2.constraints.items():
    print (f"{name}: {constraint.value()}") 

objective: 1.7733333333333334e-10
x1: 778.0
x2: 0.0
x3: 30.0
dm_cost: 0.0
dM_cost: 3990.0
dm_kits: 0.0
dM_kits: 0.0
Min_Kits_Premium: 0.0
Min_Ajuda_Habitantes: 396000.0
Max_Peso_Transporte: -4.0
Meta_Custo: 0.0
Meta_Kits: 0.0
